In [21]:
import sys
import os
import numpy as np
import qgis
from qgis.gui import *
from qgis.core import *
from qgis.utils import plugins
from PyQt5.QtCore import *
from qgis.analysis import QgsNativeAlgorithms

sys.path.append('/usr/share/qgis/python/plugins/')
sys.path.append('/usr/share/qgis/python/')

QgsApplication.setPrefixPath('/usr', True)
app = QgsApplication([], False)
app.initQgis()

import processing
from processing.core.Processing import Processing
Processing.initialize()

QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())
# for alg in QgsApplication.processingRegistry().algorithms():
#         print(alg.id(), "--->", alg.displayName())

# Stop QGIS appllication
# app.exitQgis()
# app.exit()

False

In [22]:
import itertools

In [23]:
from qgis.core import QgsProject, QgsApplication
# Get the project instance
project = QgsProject.instance()
# Print the current project file name (might be empty in case no projects have been loaded)
# print(project.fileName())

# Load  project
project.read(f"/home/luca/Code/mappy.git/input_data/qgis_project.qgz")
# project.read("/home/luca/Code/mappy.git/input_data/mars/mars.qgz")
print(project.fileName())

/home/luca/Code/mappy.git/input_data/qgis_project.qgz


In [24]:
# project.mapLayers()
# cont = project.mapLayersByName("contacts")[0]
# cont = project.mapLayersByName("closed_contact")[0]

# pols = project.mapLayersByName("map")[0]

In [25]:
ex = project.mapLayersByName("exploded")[0]
ex

<QgsVectorLayer: 'exploded' (ogr)>

In [26]:
ln1.geometry

<function QgsFeature.geometry>

In [120]:

index = QgsSpatialIndex() # Spatial index

index.addFeatures(ex.getFeatures())

True

In [121]:
p1.distance(p2)

6.617556385250989

In [122]:
def equal_segments(seg1, seg2, threshold=1e-6):
    s1_s, s1_e = seg1.vertices()
    s2_s, s2_e = seg2.vertices()
    
    
    if s1_s.distance(s2_s) < threshold and s1_e.distance(s2_e) < threshold:
        return True
    
    if s1_s.distance(s2_e) < threshold and s1_e.distance(s2_s) < threshold:
        return True
    else:
        return False

In [123]:
%%time
todel = []
for ln in ex.getFeatures():
    index.deleteFeature(ln)
    
    cands = index.intersects(ln.geometry().boundingBox())
        
    ln1 = ln.geometry()   
    for ca in cands:
        totest = ex.getFeature(ca)
        ln2 = totest.geometry()
       
        are_equal = equal_segments(ln1, ln2)
        if are_equal:
            index.deleteFeature(totest)
            todel.append(ca)
        
#     print(f"{ln.id()} cands: {cands}")

CPU times: user 4.44 s, sys: 1.6 s, total: 6.04 s
Wall time: 6.06 s


In [124]:
def save_to_geopackage( gpkg_filename,  layer,layername="layer"):
    opts = QgsVectorFileWriter.SaveVectorOptions()
    opts.driverName = "GPKG"
    opts.layerName = layername
    opts.actionOnExistingFile = QgsVectorFileWriter.CreateOrOverwriteLayer
    
    
    return QgsVectorFileWriter.writeAsVectorFormatV2(layer, gpkg_filename, QgsCoordinateTransformContext(), opts)   

In [125]:
vl = QgsVectorLayer("LineString", "temp", "memory")
pr = vl.dataProvider()

for f in ex.getFeatures():
    if f.id() not in todel:
        
        pr.addFeature(f)
vl.updateExtents() 


save_to_geopackage("ciao.gpkg", vl, "testing3")

(0, '')

In [70]:
v = a-b
np.linalg.norm(v)

/usr/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


4.927726237058912

In [12]:
for f in ex.getFeatures():
    print(f)

In [132]:
def multi_to_single_part(layer):
    o = processing.run("native:multiparttosingleparts", {"INPUT": layer, "OUTPUT": "TEMPORARY_OUTPUT"})["OUTPUT"]
    return o

In [133]:
cont = multi_to_single_part(cont)




In [142]:
def explode_in_sub_segments(line_geometry):
    verts = list(line_geometry.vertices())

    sub_lines = []
    
    for segid, ( v1, v2 ) in enumerate(zip(verts, verts[1:])):
        line = QgsGeometry.fromPolyline([v1,v2])
        line.segid = segid
        sub_lines.append(line)
        
    return sub_lines

In [143]:
out = {}
for f in cont.getFeatures():
    g = f.geometry()
    out[f.id()] = explode_in_sub_segments(g)
    

In [145]:
for fid in out.keys():
    line = out[fid]

In [148]:
line[0].segid

0

In [120]:
def find_intersections(lines):
    ints = []
    for l1, l2 in itertools.combinations(lines, 2):
        if l1.crosses(l2):
            i = l1.intersection(l2)
            ints.append(i)
            
    return ints

def find_intersections2(lines):
    ints = []
    for l1, l2 in itertools.combinations(lines, 2):
        if l1.crosses(l2):
            i = l1.intersection(l2)
            ints.append(i)
            continue
        
        
        if l1.intersects(l2):
            i = l1.intersection(l2)
            if i.type() == QgsWkbTypes.geometryType(QgsWkbTypes.LineString):
                for p in i.vertices():
                    ints.append(QgsGeometry (p))
            
    return ints

In [121]:
def find_self_intersections(line_feature):
    
    g = line_feature.geometry()
    sublines = explode_in_sub_segments(g)
    return find_intersections2(sublines)
    
        
        

In [122]:
out = []
for f in cont.getFeatures():
    ints = find_self_intersections(f)
    if len(ints) > 2:
        ints = [ints[0], ints[-1]]
    out +=  ints


In [123]:
out

[<QgsGeometry: Point (222.78929466406384563 21.66631021340212016)>,
 <QgsGeometry: Point (230.27228378488533167 19.44297040052873982)>,
 <QgsGeometry: Point (216.88297380846066176 19.67275779342391928)>,
 <QgsGeometry: Point (237.19705918755019525 22.15844848522790045)>,
 <QgsGeometry: Point (236.97271161054095501 19.32606032548467923)>,
 <QgsGeometry: Point (239.50042096200135688 21.08595473084476168)>,
 <QgsGeometry: Point (240.41739365293409492 21.35917438094644893)>,
 <QgsGeometry: Point (233.12588223301591484 18.46159058581272205)>,
 <QgsGeometry: Point (234.92066284909083151 18.46159058581272205)>]

In [124]:
vl = QgsVectorLayer("Point", "temp", "memory")
pr = vl.dataProvider()

for p in out:
    f = QgsFeature()
    f.setGeometry(p)
    pr.addFeature(f)
vl.updateExtents() 

In [125]:
save_to_geopackage("ciao.gpkg", vl, "testing")

(0, '')

In [64]:
def get_spatial_index(lines):
    index = QgsSpatialIndex() # Spatial index

    for line in lines:
        f = QgsFeature()
        f.setGeometry(line)
        index.insertFeature(f)
        
    return index

/usr/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
def explode_lines(layer):
    return processing.run("native:explodelines", {"INPUT": layer, "OUTPUT": "TEMPORARY_OUTPUT"})["OUTPUT"]



ex = explode_lines(cont)

def drop_field(layer, fieldname="fid"):
    if fieldname not in layer.fields().names():
        return False
    else:
        id = layer.fields().indexFromName(fieldname)
        layer.dataProvider().deleteAttributes([id])
        layer.updateFields()
        return True
        
# drop_field(ex)

def rename_field(layer, old, new):
    if old not in layer.fields().names():
        return False
    else:
        id = layer.fields().indexFromName(old)
        dp = ex.dataProvider()
        dp.renameAttributes({id:new})
        layer.updateFields()
        return True
           


In [14]:
cont = multi_to_single_part(cont)
rename_field(cont, "fid", "original_fid")

True

In [15]:
cont

<QgsVectorLayer: 'Single parts' (memory)>

In [16]:
# for f in ex.getFeatures():
#     print(f.id())
ex = cont

In [17]:


# save_to_geopackage("ciao.gpkg", ex, "exploded")

In [18]:
def extract_feautures(layer):
    lines = []
    counter = 1
    for f in cont.getFeatures():
        fid = f.id()
        g = f.geometry()
        
        if g.isMultipart():
            for i, p in enumerate(g.parts()):
                newf = QgsFeature(counter)
                counter+=1

                newf.setGeometry(g)
                
                
                lines.append([fid, i, newf])
        else:
            newf = QgsFeature(counter)
            counter+=1

            newf.setGeometry(g)
            lines.append([fid, None, newf])
            
    return lines

def get_spatial_index(lines):
    index = QgsSpatialIndex() # Spatial index

    for line in lines:
        index.insertFeature(line[-1])
        
    return index


def get_features_spatial_index(layer):
    index = QgsSpatialIndex() # Spatial index

    for f in layer.getFeatures():
        index.insertFeature(f)
        
    return index

In [19]:
# lines = extract_feautures(cont)
index = get_features_spatial_index(cont)

<ipython-input-18-bb8f810321db>:39: DeprecationWarning: QgsSpatialIndex.insertFeature() is deprecated
  index.insertFeature(f)


In [32]:
ints = []
for f in cont.getFeatures():
    ids = index.intersects(f.geometry().boundingBox())
    newids = []
    for fid in ids:
        thisf = cont.getFeature(fid)
        c = f.geometry().crosses(thisf.geometry())
        if c:
            ints.append(sorted([f.id(), fid]) + ["C"])
#             print("cross")
            continue
            
        c = f.geometry().intersects(thisf.geometry())
        if c:
#             print("overlay")
            ints.append(sorted([f.id(), fid]) + ["I"])
            continue

        

In [58]:
QgsWkbTypes.Point

1

In [51]:
QgsWkbTypes.Point

/usr/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1

In [59]:
out = []
for ida, idb, typ in ints:
    f1 = cont.getFeature(ida)
    f2 = cont.getFeature(idb)
#     print(f"{ida} - {idb}")
    intersection = f1.geometry().intersection(f2.geometry())
    itype = intersection.type()
    print(itype)
    print(intersection)
    if itype == QgsWkbTypes.Point:
        out.append(intersection)
    elif itype == QgsWkbTypes.MultiPoint:
        for p in intersection.parts():
            out.append(p)

0
<QgsGeometry: Point (222.58088274010100349 15.49644258108680717)>
0
<QgsGeometry: Point (222.07708508845828987 18.19429328854174699)>
1
<QgsGeometry: MultiLineString ((222.52490522325189204 15.52443133951135579, 222.40548652064026669 15.52443133951135579),(222.40548652064026669 15.52443133951135579, 222.34577716933446823 15.55428601516426212),(222.34577716933446823 15.55428601516426212, 222.31592249368154057 15.55428601516426212),(222.31592249368154057 15.55428601516426212, 222.28606781802864134 15.58414069081716846),(222.28606781802864134 15.58414069081716846, 222.13679443976411676 15.58414069081716846),(222.13679443976411676 15.58414069081716846, 222.07708508845828987 15.6139953664700748),(222.07708508845828987 15.6139953664700748, 222.04723041280539064 15.64385004212298291),(222.04723041280539064 15.64385004212298291, 222.01737573715249141 15.67370471777588925),(222.01737573715249141 15.67370471777588925, 221.95766638584666453 15.67370471777588925),(221.95766638584666453 15.673704

In [53]:
out

[<QgsGeometry: MultiLineString ((222.52490522325189204 15.52443133951135579, 222.40548652064026669 15.52443133951135579),(222.40548652064026669 15.52443133951135579, 222.34577716933446823 15.55428601516426212),(222.34577716933446823 15.55428601516426212, 222.31592249368154057 15.55428601516426212),(222.31592249368154057 15.55428601516426212, 222.28606781802864134 15.58414069081716846),(222.28606781802864134 15.58414069081716846, 222.13679443976411676 15.58414069081716846),(222.13679443976411676 15.58414069081716846, 222.07708508845828987 15.6139953664700748),(222.07708508845828987 15.6139953664700748, 222.04723041280539064 15.64385004212298291),(222.04723041280539064 15.64385004212298291, 222.01737573715249141 15.67370471777588925),(222.01737573715249141 15.67370471777588925, 221.95766638584666453 15.67370471777588925),(221.95766638584666453 15.67370471777588925, 221.92781171019376529 15.70355939342879559),(221.92781171019376529 15.70355939342879559, 221.83824768323503918 15.7035593934

In [11]:
def removeDuplicates(lst): 
    return [t for t in (set(tuple(i) for i in lst))] 

ints = removeDuplicates(ints)

In [12]:
idx = ex.fields().indexFromName('original_fid')

pts = []
for fida, fidb in ints:
    fa = ex.getFeature(fida)
    fb = ex.getFeature(fidb)
    ga = fa.geometry()
    gb = fb.geometry()
    
    
    
    

    oa = fa.attributes()[idx]
    ob = fb.attributes()[idx]
    
    
    
    if oa != ob:
        pass
    
    if 1:
        
        pt = ga.intersection(gb)
        pts.append(pt)
    
    

In [13]:
f.attributes()

[47, NULL]

In [14]:
vl = QgsVectorLayer("Point", "temp", "memory")
pr = vl.dataProvider()
for pt in pts:
    f = QgsFeature()
    f.setGeometry(pt)
    pr.addFeature(f)
    
    
    
    
# f = QgsFeature()
# f.setGeometry(QgsGeometry.fromPointXY(QgsPointXY(10,10)))
# f.setAttributes(["Ada L.", 2, 0.3])
# pr.addFeature(f)
vl.updateExtents() 

In [15]:
save_to_geopackage("ciao.gpkg", vl, "ints")

(0, '')

In [14]:
g = f.geometry()
# g.splitGeometry?

In [15]:
raise

RuntimeError: No active exception to reraise

In [ ]:
cont.selectAll()
cont = processing.run("native:saveselectedfeatures", {'INPUT': cont, 'OUTPUT': 'TEMPORARY_OUTPUT'})['OUTPUT']

pols.selectAll()
pols = processing.run("native:saveselectedfeatures", {'INPUT': pols, 'OUTPUT': 'TEMPORARY_OUTPUT'})['OUTPUT']

In [ ]:
dissolved = processing.run("native:dissolve", {'INPUT':pols,'FIELD':[],'OUTPUT':'TEMPORARY_OUTPUT'})["OUTPUT"]

dissolved #QgsProcessingUtils.generateTempFilename("dissolved.gpkg")}

In [ ]:
ex = processing.run("native:polygonfromlayerextent", {"INPUT":cont, "OUTPUT":"TEMPORARY_OUTPUT"})["OUTPUT"]

ex1 = processing.run("native:buffer",  {"INPUT":  ex, "DISTANCE":1e-2, "OUTPUT":"TEMPORARY_OUTPUT"})["OUTPUT"]


mask = processing.run("native:difference", {'INPUT':ex1,'OVERLAY':dissolved,'OUTPUT':'TEMPORARY_OUTPUT'})["OUTPUT"]

In [ ]:
mask.featureCount()

In [ ]:
ff = list(mask.getFeatures())
[f.geometry() for f in ff]

In [ ]:
f = mask.getFeature(1)


print(pols.addFeature(f, QgsFeatureSink.FastInsert))



In [ ]:
pols.fields()

In [ ]:
# [2021-02-21 11:20:37] processing.run("native:dissolve", {'INPUT':'','FIELD':[],'OUTPUT':'TEMPORARY_OUTPUT'})
# dis = processing.run("native:dissolve", {"INPUT": pols, "OUTPUT":"TEMPORARY_OUTPUT"})


In [ ]:
index = QgsSpatialIndex() # Spatial index

for ft in pols.getFeatures():
    
    index.insertFeature(ft)


In [ ]:


# pars = {'INPUT':cont2.id(),'OVERLAY':dissolved,'INPUT_FIELDS':[],'OVERLAY_FIELDS':[],'OVERLAY_FIELDS_PREFIX':'','OUTPUT':'TEMPORARY_OUTPUT'}
# cont = processing.run("native:intersection", pars)["OUTPUT"]

In [ ]:
# QgsVectorFileWriter.writeAsVectorFormat(cont,f"output_pols",'utf-8',driverName='ESRI Shapefile')    

In [ ]:
outtable = []

for f in cont.getFeatures():
    fid = f.id()
    
    g = f.geometry()
    if g.isEmpty() or not f.isValid() or g.length() < 1e-16:
        continue
        
    
    for pid, p in enumerate(g.parts()):
        start = p[0]
        end = p[-1]

        outtable.append([fid, pid, [start, end]])
            
        
        


In [ ]:
def find_polygons_containing_point(pt, index, polygons):
    ids = index.intersects(pt.boundingBox()) # polygons intersecting the point
    pp= polygons.getFeatures(ids)
    real = []
    for p, i in zip(pp, ids):
        if p.geometry().contains(QgsPointXY(pt)):
            real.append(p.id())
            
    return real
            

    

In [ ]:
exteriors = []
cuttable = []
for fid, pid, (start, end) in outtable:
    mypols = []
    pstart = find_polygons_containing_point(start, index, pols)
    pend = find_polygons_containing_point(end, index, pols)
    mypols += pend + pstart
    
    if len(mypols) >1:
        cuttable.append([fid, pid, mypols])
        
    elif len(mypols) < 2:
        exteriors.append([fid, pid])

In [ ]:
def get_sel(layer):
    return processing.run("native:saveselectedfeatures", {'INPUT':layer.id(),'OUTPUT':'TEMPORARY_OUTPUT'})["OUTPUT"]

In [ ]:
# QgsProcessingFeatureSourceDefinition?

In [ ]:
def clip_with_selection(lines, polygons):
    pars = {"INPUT":QgsProcessingFeatureSourceDefinition(lines.id(), True),
            "OVERLAY": QgsProcessingFeatureSourceDefinition(polygons.id(), True),
            "OUTPUT":"memory:"}
    
    
    return processing.run("native:difference", pars)

def clip_with_selection2(lines, polygons):
    lin = processing.run("native:saveselectedfeatures", {'INPUT':lines,'OUTPUT':'TEMPORARY_OUTPUT'})["OUTPUT"]
    pol = processing.run("native:saveselectedfeatures", {'INPUT':polygons,'OUTPUT':'TEMPORARY_OUTPUT'})["OUTPUT"]
    
    
    pars = {"INPUT":lin,
            "OVERLAY": pol,
            "OUTPUT":'TEMPORARY_OUTPUT'}
    
    
    return processing.run("native:difference", pars)["OUTPUT"]

def clip_external(lines, dissolved):
    lin = processing.run("native:saveselectedfeatures", {'INPUT':lines,'OUTPUT':'TEMPORARY_OUTPUT'})["OUTPUT"]
    return processing.run("native:clip", {"INPUT": lin, "OVERLAY":dissolved, "OUTPUT":"TEMPORARY_OUTPUT"})["OUTPUT"]
    
    

In [ ]:
for i, (lineid, subid, polygons) in enumerate(cuttable):
    pols.removeSelection()
    cont.removeSelection()
    [pols.select(p) for p in polygons]
    cont.select(lineid)
    
    o = clip_with_selection2(cont, pols)
    
    if o.featureCount() > 0:
        f = o.getFeature(1)
        gg = f.geometry()

        cont.startEditing()
        cont.changeGeometry(lineid,gg)
        cont.commitChanges()

In [ ]:
for i, (lineid, subid) in enumerate(exteriors):
    dissolved.removeSelection()
    cont.removeSelection()
    cont.select(lineid)
    
    o = clip_external(cont, dissolved)
    if o.featureCount() > 0:
        f = o.getFeature(1)
        gg = f.geometry()

        cont.startEditing()
        cont.changeGeometry(lineid,gg)
        cont.commitChanges()
    

In [ ]:
cont

In [ ]:
QgsVectorFileWriter.writeAsVectorFormat(cont,f"output_pols",'utf-8',driverName='ESRI Shapefile')    

In [ ]:
# QgsVectorFileWriter.writeAsVectorFormat(cont,"output",'utf-8',driverName='ESRI Shapefile')

In [ ]:
l = cont.getFeature(36)
p = pols.getFeature(30)
a

In [ ]:
pols.removeSelection()

In [ ]:
cont.select(36)
pols.select(30)


In [ ]:
pols.selectedFeatureIds()

In [ ]:
print([a.id() for a in QgsApplication.processingRegistry().algorithms() if "clip" in a.id()])

In [ ]:
[2021-02-15 11:39:59] processing.run("native:clip", {'INPUT':QgsProcessingFeatureSourceDefinition('contacts_7768adb1_5e8e_4a87_a7b2_aebd7bb4303e', True),
                                                     'OVERLAY':QgsProcessingFeatureSourceDefinition('map_62aaeba8_b51e_4242_82bc_ea14f8a87fbe', True),'OUTPUT':'TEMPORARY_OUTPUT'})